In [356]:
! pip3 install -U selenium
! pip3 install webdriver-manager
! pip3 install pandas 
! pip3 install requests
! pip3 install numpy
! pip3 install bs4
! pip3 install html
! pip3 install tqdm
! pip3 install gspread
! pip3 install df2gspread
! pip3 install time
! pip3 install chromedriver_autoinstaller
! pip3 install fake_useragent
! pip3 install datetime
! pip3 install undetected_chromedriver
! pip3 install webdriver_manager
! pip3 install warnings
! pip3 install geopy

  Using cached html-1.16.tar.gz (7.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/dm/p_gv33kn0vl5wg11gq43jzvh0000gn/T/pip-install-5o6wp0nu/html_e6135459345f4bdb9064b7ae0463f062/setup.py", line 12, in <module>
          long_description = __doc__.decode('utf8'),
      AttributeError: 'str' object has no attribute 'decode'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached argparse-1.4.0-p

In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import html
from tqdm import tqdm
import os
import gspread
import df2gspread as d2g
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
import chromedriver_autoinstaller as chromedriver
chromedriver.install()

'/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/chromedriver_autoinstaller/107/chromedriver'

In [3]:
!pip install --upgrade --force-reinstall chromedriver-binary-auto
import chromedriver_binary

  Using cached chromedriver_binary_auto-0.2.1-py3-none-any.whl
  Attempting uninstall: chromedriver-binary-auto
    Found existing installation: chromedriver-binary-auto 0.2.1
    Uninstalling chromedriver-binary-auto-0.2.1:
      Successfully uninstalled chromedriver-binary-auto-0.2.1


In [4]:
from fake_useragent import UserAgent
import datetime
from selenium import webdriver
from xvfbwrapper import Xvfb
from selenium.webdriver.chrome.options import Options

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import undetected_chromedriver

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0")
options.add_argument("--disable-blink-features=AutomationControlled")

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")
driver.close()



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 107.0.5304
INFO:WDM:Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
INFO:WDM:Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache
INFO:WDM:Driver [/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


In [8]:
#try:
#    driver = undetected_chromedriver.Chrome()
#    # driver.get("https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html")
#    driver.get("https://www.cian.ru/sale/flat/263173912/")
#    time.sleep(15)
#    page_source = driver.page_source
#except Exception as ex:
#    print(ex)
#finally:
#    driver.close()
#    driver.quit()

In [9]:
def get_html(url):
    req = requests.get(url, headers={"User-Agent": UserAgent().chrome})
    return req.text.encode(req.encoding)

In [10]:
def get_title(soup):
    try:
        title = soup.find("h1").text.strip()
    except Exception as e:
        #print(str(e) + " title")
        title = "Не указано"
    return title


def get_address(soup):
    try:
        address = soup.find("address").text.strip()
        if "На карте" in address:
            address = address[:address.rfind("На карте")]
        # separating data from the address string
        district, street = "Не указано", "Не указано"
        city = address.split(",")[1].strip()
        block_number = address.split(",")[-1].strip()
        if "ул " in block_number.lower() or "ул." in block_number.lower() or "улица" in block_number.lower() \
                or " пер" in block_number.lower() or "проезд" in block_number.lower() or "проспект" in block_number.lower():
            street = block_number
            block_number = "Не указано"

        for param in address.split(",")[1:-1]:
            if "ул " in param.lower() or "ул." in param.lower() or "улица" in param.lower() or " пер" in param.lower() \
                    or "проезд" in param.lower() or "проспект" in param.lower():
                street = param.strip()
            elif "район" in param.lower() or "р-н" in param.lower():
                district = param.strip()

        if street.split()[-1].strip().isdigit():
            block_number = street.split()[-1].strip()
            street = " ".join(street.split()[:-1]).strip()

        return city, district, street, block_number
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_title\n")
    return ["Не указано"] * 4


def get_price(soup):
    try:
        price = soup.find("span", {"itemprop": "price"})
        if price is not None:
            price = price.text.strip()
        else:
            price = "от " + soup.find("span", {"itemprop": "lowPrice"}).text.strip() + \
                    " до " + soup.find("span", {"itemprop": "highPrice"}).text.strip() + "/мес."
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_price\n")
        price = "Не указано"
    return price


def get_selling_type(soup):
    try:
        paragraphs = [x for x in soup.find_all("p") if x.get("class") is not None
                      and len(x.get("class")) == 1 and "description--" in x.get("class")[0]]
        if paragraphs:
            selling_type = paragraphs[0].text.strip()
        else:
            selling_type = "Не указано"
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_selling_type\n")
        selling_type = "Не указано"
    return selling_type


def get_seller_type(soup):
    try:
        divs = [x for x in soup.find_all("div") if x.get("class") is not None
                and len(x.get("class")) == 1 and "honest-container" in x.get("class")[0]]
        if not divs:
            seller_type = "Не указано"
        else:
            seller_type = divs[0].text.strip()
            if seller_type is not None and seller_type.lower() == "собственник":
                seller_type = "Собственник"
            else:
                seller_type = "Посредник"
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_seller_type\n")
        seller_type = "Не указано"
    return seller_type


def get_seller_name(soup):
    try:
        name = [x for x in soup.find_all("h2") if x.get("class") is not None and len(x.get("class")) == 1
                and "title--" in x.get("class")[0]]
        if name:
            name = name[0].text.strip()
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_seller_name\n")
        name = "Не указано"
    return name


def get_photos(url):
    try:
        driver = webdriver.Chrome()
        driver.get(url)

        images = []
        images_list = driver.find_elements_by_class_name("fotorama__img")
        images_list = [x.get_attribute("src") for x in images_list if "-2." in x.get_attribute("src")]
        for image in images_list:
            link = image.replace("-2.", "-1.")
            images.append(link)
        images = "\n".join(images)
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_photos\n")
        images = "Не указано"
    return images


def get_description(soup):
    try:
        paragraphs = [x for x in soup.find_all("p") if x.get("class") is not None
                      and len(x.get("class")) == 1 and "description-text--" in x.get("class")[0]]
        description = paragraphs[0].text.strip()
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_description\n")
        description = "Не указано"
    return description


def get_date(soup):
    try:
        date = soup.find("div", id="frontend-offer-card").find("main").find_all("div")[4].text.strip()
        if "вчера" in date:
            date = str(datetime.datetime.today() - datetime.timedelta(days=1)).split()[0]
        elif "сегодня" in date:
            date = str(datetime.datetime.today()).split()[0]
        else:
            date = "too old"
    except Exception as e:
        with open("logs.txt", "a", encoding="utf8") as file:
            file.write(str(e) + " cian get_date\n")
        date = "Не указано"
    return date


In [11]:
def find_kitchen(soup):
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')
    j = 0
    flag = 0
    for i in temp_2:
        if i.text == 'Кухня':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_1[ind].text)

In [12]:
def find_floor(soup):
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')
    j = 0
    flag = 0
    for i in temp_2:
        if i.text == 'Этаж':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_1[ind].text)

In [13]:
def find_finishing(soup):
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')
    j = 0
    flag = 0
    for i in temp_2:
        if i.text == 'Отделка':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_1[ind].text)

In [14]:
def find_living(soup):
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')
    j = 0
    flag = 0
    for i in temp_2:
        if i.text == 'Жилая':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_1[ind].text)

In [135]:
def find_year(soup):
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')
    j = 0
    flag = 0
    for i in temp_2:
        if i.text == 'Построен':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_1[ind].text)

In [147]:
def type_floar(soup):
    temp_3 = soup.find_all('span', 'a10a3f92e9--name--x7_lt')
    temp_4 = soup.find_all('span','a10a3f92e9--value--Y34zN')
    j = 0
    flag = 0
    for i in temp_3:
        if i.text == 'Тип жилья':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_4[ind].text)

In [148]:
def type_san(soup):
    temp_3 = soup.find_all('span', 'a10a3f92e9--name--x7_lt')
    temp_4 = soup.find_all('span','a10a3f92e9--value--Y34zN')
    j = 0
    flag = 0
    for i in temp_3:
        if i.text == 'Санузел':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_4[ind].text)

In [149]:
def type_bal(soup):
    temp_3 = soup.find_all('span', 'a10a3f92e9--name--x7_lt')
    temp_4 = soup.find_all('span','a10a3f92e9--value--Y34zN')
    j = 0
    flag = 0
    for i in temp_3:
        if i.text == 'Балкон/лоджия' or i.text == 'Балкон':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_4[ind].text)

In [150]:
def type_rem(soup):
    temp_3 = soup.find_all('span', 'a10a3f92e9--name--x7_lt')
    temp_4 = soup.find_all('span','a10a3f92e9--value--Y34zN')
    j = 0
    flag = 0
    for i in temp_3:
        if i.text == 'Ремонт':
            ind = j
            flag = 1
        j += 1
    if flag == 0:
        return('Не указано')
    else:
        return(temp_4[ind].text)

In [377]:
def find_distance(soup):
    temp_5 = soup.find_all('span', 'a10a3f92e9--highway_distance--wUNBn')
    if len(temp_5) == 0:
        temp_5 = soup.find_all('span', 'a10a3f92e9--underground_time--iOoHy')
        temp_5_2 = soup.find_all('a', 'a10a3f92e9--underground_link--Sxo7K')
        if len(temp_5_2) == 0 or len(temp_5) == 0:
            return np.nan
        else: 
            if (temp_5_2[0].text + temp_5[0].text).find('МКАД') != -1:
                return np.nan
            else:
                return float((temp_5_2[0].text + temp_5[0].text).split('⋅  ')[1].split('мин')[0])
    else:
        return np.nan

# Скачать ВСЁ

In [348]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
#C:\\WebDrivers\\chromedriver.exe' для винды
driver = undetected_chromedriver.Chrome()
driver.get('https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&sort=creation_date_desc')
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
temp_7 = soup.find_all('a', '_93444fe79c--link--eoxce')
to_iter = int(temp_7[0]['href'].split('/')[-2])
#options = webdriver.ChromeOptions()
#options.add_argument("start-maximized")
#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)
#driver = webdriver.Chrome(chrome_options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
#driver.get("https://sslproxies.org/")
#driver = webdriver.Chrome()
#driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
#ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
#ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
#driver.quit()
driver.close()
driver = undetected_chromedriver.Chrome()
resp = requests.get('https://free-proxy-list.net/') 
df = pd.read_html(resp.text)[0]
proxies = []
for i in range(0,len(df)):
    proxies.append(str(df['IP Address'][i])+':'+str(df['Port'][i]))
j = -1
df = pd.DataFrame(columns=['id','Link','Date','Address','Число комнат','Цена','Общая площадь','Жилая площадь кв',
                          'Площадь кухни','Этаж','Этажей в доме', 'Отделка','Расстояние до метро','Год постройки', 'Тип жилья',
                          'Санузел','Балкон','Ремонт'])

k = 0 
for i in tqdm(range(0,to_iter)):
    j += 1
    if j > len(proxies):
        j = 0
#        try:
#            print("Proxy selected: {}".format(proxies[i]))
#            options = webdriver.ChromeOptions()
#            options.add_argument('--proxy-server={}'.format(proxies[i]))
#            driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
#            driver.get("https://www.whatismyip.com/proxy-check/?iref=home")
#            if "Proxy Type" in WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "p.card-text"))):
#                break
#        except Exception:
#           driver.quit()

    url = 'https://www.cian.ru/sale/flat/'+str(i)
    #options = webdriver.ChromeOptions()
    #options.add_argument('--proxy-server={}'.format(proxies[j]))
    #driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
    driver.get(url)
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    temp_6 = soup.find_all('div', 'a10a3f92e9--container--RXoIe')
    temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
    #print(soup.find('title').text.split()[0])
    if soup.find('title').text.split()[0] == 'Ошибка':
        continue
    elif temp.text.split()[0].split('-')[0] == 'Участок,':
        continue
    elif soup.find("address").text.strip(',').split()[0] != 'Москва':
        continue
    elif temp_6[0].text == 'Объявление снято с публикации':
        continue
    else:
        df.loc[ len(df.index )] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan,np.nan]
        df['id'][k] = i
        df['Link'][k] = url
        df['Date'][k] = get_date(soup)
        df['Address'][k] = soup.find("address").text.strip()
        temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
        df['Число комнат'][k] = temp.text.split()[0].split('-')[0]
        df['Цена'][k] = get_price(soup).replace('\xa0','').replace('₽', '')
        temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
        df['Общая площадь'][k] = temp.text.split()[2].replace(',', '.')
        df['Расстояние до метро'][k] = find_distance(soup)
        
        if find_living(soup) == 'Не указано':
            df['Жилая площадь кв'][k] = 'Не указано'
        else: 
            df['Жилая площадь кв'][k] = find_living(soup).split()[0].replace(',', '.')
        if find_kitchen(soup) == 'Не указано':
            df['Площадь кухни'][k] = 'Не указано'
        else: 
            df['Площадь кухни'][k] = find_kitchen(soup).split()[0].replace(',', '.')
        if find_floor(soup) == 'Не указано':
            df['Этаж'][k] = 'Не указано'
        else:
            df['Этаж'][k] = (find_floor(soup).split()[0])
                          
        if find_floor(soup) == 'Не указано':
            df['Этажей в доме'][k] = 'Не указано'
        else: 
            df['Этажей в доме'][k] = find_floor(soup).split()[2]
        if find_finishing(soup) == 'Не указано':
            df['Отделка'][k] = 'Не указано'
        else:
            df['Отделка'][k] = find_finishing(soup)
        if find_year(soup) == 'Не указано':
            df['Год постройки'][k] = 'Не указано'
        else:
            df['Год постройки'][k] = find_year(soup)
        if type_floar(soup) == 'Не указано':
            df['Тип жилья'][k] = 'Не указано'
        else:
            df['Тип жилья'][k] = type_floar(soup)
        if type_san(soup) == 'Не указано':
            df['Санузел'][k] = 'Не указано'
        else:
            df['Санузел'][k] = type_san(soup)
        if type_bal(soup) == 'Не указано':
            df['Балкон'][k] = 'Не указано'
        else:
            df['Балкон'][k] = type_bal(soup)
        if type_rem(soup) == 'Не указано':
            df['Ремонт'][k] = 'Не указано'
        else:
            df['Ремонт'][k] = type_rem(soup)
        k += 1

Exception ignored in: <function Chrome.__del__ at 0x7fed14b6c550>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/undetected_chromedriver/__init__.py", line 657, in __del__
    self.quit()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/undetected_chromedriver/__init__.py", line 634, in quit
    shutil.rmtree(self.user_data_dir, ignore_errors=False)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/shutil.py", line 740, in rmtree
    return _rmtree_unsafe(path, onerror)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/shutil.py", line 613, in _rmtree_unsafe
    _rmtree_unsafe(fullname, onerror)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/shutil.py", line 613, in _rmtree_unsafe
    _rmtree_unsafe(fullname, onerror)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/shutil.py", line 613

In [ ]:
df_all.to_excel('df_all_all.xlsx')

In [270]:
driver = undetected_chromedriver.Chrome()
driver.get('https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&sort=creation_date_desc')
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
temp_7 = soup.find_all('a', '_93444fe79c--link--eoxce')
hrefs = []

In [271]:
for i in range(0,len(temp_7)):
    hrefs.append(temp_7[i]['href'])

In [272]:
hrefs

['https://www.cian.ru/sale/flat/279800615/',
 'https://www.cian.ru/sale/flat/279800556/',
 'https://www.cian.ru/sale/flat/279800554/',
 'https://www.cian.ru/sale/flat/279800523/',
 'https://www.cian.ru/sale/flat/279800458/',
 'https://www.cian.ru/sale/flat/279800445/',
 'https://www.cian.ru/sale/flat/279800429/',
 'https://www.cian.ru/sale/flat/279800342/',
 'https://www.cian.ru/sale/flat/279800323/',
 'https://www.cian.ru/sale/flat/279800294/',
 'https://www.cian.ru/sale/flat/279800285/',
 'https://www.cian.ru/sale/flat/279800259/',
 'https://www.cian.ru/sale/flat/279800206/',
 'https://www.cian.ru/sale/flat/279800199/',
 'https://www.cian.ru/sale/flat/279800198/',
 'https://www.cian.ru/sale/flat/279799826/',
 'https://www.cian.ru/sale/flat/279799817/',
 'https://www.cian.ru/sale/flat/279799784/',
 'https://www.cian.ru/sale/flat/279799738/',
 'https://www.cian.ru/sale/flat/279799730/',
 'https://www.cian.ru/sale/flat/279799729/',
 'https://www.cian.ru/sale/flat/279799710/',
 'https://

In [273]:
driver.close()
driver = undetected_chromedriver.Chrome()

In [284]:
url = hrefs[1]
driver.get(url)
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
temp_5 = soup.find_all('span', 'a10a3f92e9--highway_distance--wUNBn')
temp_5_1 = soup.find_all('span', 'a10a3f92e9--underground_time--iOoHy')
#tmpp = find_distance(soup)

In [355]:
temp_5_2 = soup.find_all('a', 'a10a3f92e9--underground_link--Sxo7K')
float((temp_5_2[0].text + temp_5_1[0].text).split('⋅  ')[1].split('мин')[0])

8.0

# Скачать первую страницу

In [378]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
#C:\\WebDrivers\\chromedriver.exe' для винды
driver = undetected_chromedriver.Chrome()
driver.get('https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&sort=creation_date_desc')
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
temp_7 = soup.find_all('a', '_93444fe79c--link--eoxce')
hrefs = []
for i in range(0,len(temp_7)):
    hrefs.append(temp_7[i]['href'])
#options = webdriver.ChromeOptions()
#options.add_argument("start-maximized")
#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)
#driver = webdriver.Chrome(chrome_options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
#driver.get("https://sslproxies.org/")
#driver = webdriver.Chrome()
#driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
#ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
#ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
#driver.quit()
driver.close()
driver = undetected_chromedriver.Chrome()
resp = requests.get('https://free-proxy-list.net/') 
df = pd.read_html(resp.text)[0]
proxies = []
for i in range(0,len(hrefs)):
    proxies.append(str(df['IP Address'][i])+':'+str(df['Port'][i]))
j = -1
df = pd.DataFrame(columns=['id','Link','Date','Address','Location','Число комнат','Цена','Общая площадь','Жилая площадь кв',
                          'Площадь кухни','Этаж','Этажей в доме', 'Отделка','Расстояние до метро','Год постройки', 'Тип жилья',
                          'Санузел','Балкон','Ремонт','Цена за метр'])

k = 0 
for i in tqdm(range(0,len(hrefs))):
    j += 1
    if j > len(proxies):
        j = 0
#        try:
#            print("Proxy selected: {}".format(proxies[i]))
#            options = webdriver.ChromeOptions()
#            options.add_argument('--proxy-server={}'.format(proxies[i]))
#            driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
#            driver.get("https://www.whatismyip.com/proxy-check/?iref=home")
#            if "Proxy Type" in WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "p.card-text"))):
#                break
#        except Exception:
#           driver.quit()

    url = hrefs[i]
    #options = webdriver.ChromeOptions()
    #options.add_argument('--proxy-server={}'.format(proxies[j]))
    #driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
    driver.get(url)
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
    #print(soup.find('title').text.split()[0])
    if soup.find('title').text.split()[0] == 'Ошибка':
        continue
    elif temp.text.split()[0].split('-')[0] == 'Участок,':
        continue
#    if soup.find("address").text.strip(',').split()[0] != 'Москва':
#        continue
#    temp_6 = soup.find_all('div', 'a10a3f92e9--container--RXoIe')
#    if temp_6[0].text == 'Объявление снято с публикации':
#        continue
    else:
        df.loc[ len(df.index )] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
        df['id'][k] = i
        df['Link'][k] = url
        df['Date'][k] = get_date(soup)
        df['Address'][k] = soup.find("address").text.strip()
        temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
        df['Число комнат'][k] = temp.text.split()[0].split('-')[0]
        df['Цена'][k] = get_price(soup).replace('\xa0','').replace('₽', '')
        temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
        df['Общая площадь'][k] = temp.text.split()[2].replace(',', '.')
        df['Расстояние до метро'][k] = find_distance(soup)
        df['Location'][k] = np.nan
        df['Цена за метр'][k] =  float(df['Цена'][k])/float(df['Общая площадь'][k])
        
        if find_living(soup) == 'Не указано':
            df['Жилая площадь кв'][k] = 'Не указано'
        else: 
            df['Жилая площадь кв'][k] = find_living(soup).split()[0].replace(',', '.')
        if find_kitchen(soup) == 'Не указано':
            df['Площадь кухни'][k] = 'Не указано'
        else: 
            df['Площадь кухни'][k] = find_kitchen(soup).split()[0].replace(',', '.')
        if find_floor(soup) == 'Не указано':
            df['Этаж'][k] = 'Не указано'
        else:
            df['Этаж'][k] = (find_floor(soup).split()[0])
                          
        if find_floor(soup) == 'Не указано':
            df['Этажей в доме'][k] = 'Не указано'
        else: 
            df['Этажей в доме'][k] = find_floor(soup).split()[2]
        if find_finishing(soup) == 'Не указано':
            df['Отделка'][k] = 'Не указано'
        else:
            df['Отделка'][k] = find_finishing(soup)
        if find_year(soup) == 'Не указано':
            df['Год постройки'][k] = 'Не указано'
        else:
            df['Год постройки'][k] = find_year(soup)
        if type_floar(soup) == 'Не указано':
            df['Тип жилья'][k] = 'Не указано'
        else:
            df['Тип жилья'][k] = type_floar(soup)
        if type_san(soup) == 'Не указано':
            df['Санузел'][k] = 'Не указано'
        else:
            df['Санузел'][k] = type_san(soup)
        if type_bal(soup) == 'Не указано':
            df['Балкон'][k] = 'Не указано'
        else:
            df['Балкон'][k] = type_bal(soup)
        if type_rem(soup) == 'Не указано':
            df['Ремонт'][k] = 'Не указано'
        else:
            df['Ремонт'][k] = type_rem(soup)
        k += 1

100%|█████████████████████████████████| 28/28 [06:41<00:00, 14.33s/it]


In [379]:
df

,id,Link,Date,Address,Location,Число комнат,Цена,Общая площадь,Жилая площадь кв,Площадь кухни,Этаж,Этажей в доме,Отделка,Расстояние до метро,Год постройки,Тип жилья,Санузел,Балкон,Ремонт,Цена за метр
0,0.0,https://www.cian.ru/sale/flat/279804323/,2022-11-06,"Москва, ЮАО, р-н Даниловский, бул. Братьев Вес...",NaN,4,57000000,95.8,63.4,19,13,20,Не указано,7.0,Не указано,Вторичка,2 совмещенных,Не указано,Дизайнерский,594989.561587
1,1.0,https://www.cian.ru/sale/flat/279804310/,2022-11-06,"Москва, СВАО, р-н Ростокино, ул. Вильгельма Пи...",NaN,3,15500000,63,35.1,7,1,5,Не указано,11.0,1955,Вторичка,1 раздельный,Не указано,Косметический,246031.746032
2,2.0,https://www.cian.ru/sale/flat/279804208/,2022-11-06,"Москва, САО, р-н Аэропорт, Новый Зыковский про...",NaN,2,14200000,44,28,6,5,5,Не указано,15.0,1961,Вторичка,1 раздельный,1 балкон,Дизайнерский,322727.272727
3,3.0,https://www.cian.ru/sale/flat/279803952/,2022-11-06,"Москва, ЮЗАО, р-н Северное Бутово, ул. Грина, ...",NaN,3,20900000,76.8,Не указано,Не указано,10,12,Не указано,NaN,2002,Вторичка,2 совмещенных,1 балкон,Евроремонт,272135.416667
4,4.0,https://www.cian.ru/sale/flat/279803944/,2022-11-06,"Москва, НАО (Новомосковский), Сосенское поселе...",NaN,2,14400000,59,30,12,5,12,Не указано,NaN,2019,Вторичка,Не указано,Не указано,Без ремонта,244067.796610
5,5.0,https://www.cian.ru/sale/flat/279803890/,2022-11-06,"Москва, ЗАО, р-н Можайский, Витебская ул., 5На...",NaN,3,22000000,73.7,44.8,10.5,1,17,Не указано,10.0,1995,Вторичка,Не указано,Не указано,Косметический,298507.462687
6,6.0,https://www.cian.ru/sale/flat/279803888/,2022-11-06,"Москва, СВАО, р-н Свиблово, Сельскохозяйственн...",NaN,1,18500000,43.5,20,12,2,23,Не указано,7.0,2020,Вторичка,Не указано,1 балкон,Дизайнерский,425287.356322
7,7.0,https://www.cian.ru/sale/flat/279803807/,2022-11-06,"Москва, ЮАО, р-н Орехово-Борисово Южное, Гурье...",NaN,3,17800000,72,43.1,10.5,1,17,Не указано,11.0,1986,Вторичка,1 раздельный,1 балкон,Косметический,247222.222222
8,8.0,https://www.cian.ru/sale/flat/279803796/,2022-11-06,"Москва, ЗАО, р-н Солнцево, ул. Богданова, 48К1...",NaN,3,21000000,75.2,46.1,10.2,3,16,Не указано,NaN,1984,Вторичка,1 совмещенный,2 лоджии,Евроремонт,279255.319149
9,9.0,https://www.cian.ru/sale/flat/279803777/,2022-11-06,"Москва, ЗАО, р-н Фили-Давыдково, ул. Ивана Фра...",NaN,2,17500000,60.5,Не указано,10.2,24,29,Не указано,5.0,2022,Вторичка Апартаменты,1 совмещенный,Не указано,Без ремонта,289256.198347


In [304]:
df = pd.DataFrame(columns=['id','Link','Date','Address','Число комнат','Цена','Общая площадь','Жилая площадь кв',
                          'Площадь кухни','Этаж','Этажей в доме', 'Отделка','Расстояние до метро','Год постройки', 'Тип жилья',
                          'Санузел','Балкон','Ремонт','Цена за метр'])
df.loc[ len(df.index )] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

In [312]:
df.loc[ len(df.index )] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

In [313]:
df

,id,Link,Date,Address,Число комнат,Цена,Общая площадь,Жилая площадь кв,Площадь кухни,Этаж,Этажей в доме,Отделка,Расстояние до метро,Год постройки,Тип жилья,Санузел,Балкон,Ремонт,Цена за метр
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Скачать первые 30 страниц

In [389]:
end_page = 3
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
#C:\\WebDrivers\\chromedriver.exe' для винды
df_all = pd.DataFrame(columns=['id','Link','Date','Address','Location','Число комнат','Цена','Общая площадь','Жилая площадь кв',
                          'Площадь кухни','Этаж','Этажей в доме', 'Отделка','Расстояние до метро','Год постройки', 'Тип жилья',
                          'Санузел','Балкон','Ремонт','Цена за метр'])

for h in tqdm(range(1,end_page)):
    driver = undetected_chromedriver.Chrome()
    driver.get('https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p='+str(h)+'&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&sort=creation_date_desc')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    temp_7 = soup.find_all('a', '_93444fe79c--link--eoxce')
    hrefs = []
    for i in range(0,len(temp_7)):
        hrefs.append(temp_7[i]['href'])
    #options = webdriver.ChromeOptions()
    #options.add_argument("start-maximized")
    #options.add_experimental_option("excludeSwitches", ["enable-automation"])
    #options.add_experimental_option('useAutomationExtension', False)
    #driver = webdriver.Chrome(chrome_options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
    #driver.get("https://sslproxies.org/")
    #driver = webdriver.Chrome()
    #driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
    #ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
    #ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
    #driver.quit()
    driver.close()
    driver = undetected_chromedriver.Chrome()
    resp = requests.get('https://free-proxy-list.net/') 
    df = pd.read_html(resp.text)[0]
    proxies = []
    for i in range(0,len(hrefs)):
        proxies.append(str(df['IP Address'][i])+':'+str(df['Port'][i]))
        j = -1 
    k = 0
    df = pd.DataFrame(columns=['id','Link','Date','Address','Location','Число комнат','Цена','Общая площадь','Жилая площадь кв',
                          'Площадь кухни','Этаж','Этажей в доме', 'Отделка','Расстояние до метро','Год постройки', 'Тип жилья',
                          'Санузел','Балкон','Ремонт','Цена за метр'])
    for i in tqdm(range(0,len(hrefs))):
        j += 1
        if j > len(proxies):
            j = 0
    #        try:
    #            print("Proxy selected: {}".format(proxies[i]))
    #            options = webdriver.ChromeOptions()
    #            options.add_argument('--proxy-server={}'.format(proxies[i]))
    #            driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
    #            driver.get("https://www.whatismyip.com/proxy-check/?iref=home")
    #            if "Proxy Type" in WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "p.card-text"))):
    #                break
    #        except Exception:
    #           driver.quit()

        url = hrefs[i]
        #options = webdriver.ChromeOptions()
        #options.add_argument('--proxy-server={}'.format(proxies[j]))
        #driver = webdriver.Chrome(options=options,executable_path='/Users/Nikon/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver')
        driver.get(url)
        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
        #print(soup.find('title').text.split()[0])
        if soup.find('title').text.split()[0] == 'Ошибка':
            continue
        elif temp.text.split()[0].split('-')[0] == 'Участок,':
            continue
    #    if soup.find("address").text.strip(',').split()[0] != 'Москва':
    #        continue
    #    temp_6 = soup.find_all('div', 'a10a3f92e9--container--RXoIe')
    #    if temp_6[0].text == 'Объявление снято с публикации':
    #        continue
        else:
            df.loc[ len(df.index )] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
            df['id'][k] = i
            df['Link'][k] = url
            df['Date'][k] = get_date(soup)
            df['Address'][k] = soup.find("address").text.strip()
            temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
            df['Число комнат'][k] = temp.text.split()[0].split('-')[0]
            df['Цена'][k] = get_price(soup).replace('\xa0','').replace('₽', '')
            temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')
            df['Общая площадь'][k] = temp.text.split()[2].replace(',', '.')
            df['Расстояние до метро'][k] = find_distance(soup)
            df['Location'][k] = np.nan
            df['Цена за метр'][k] =  float(df['Цена'][k])/float(df['Общая площадь'][k])
        
            if find_living(soup) == 'Не указано':
                df['Жилая площадь кв'][k] = 'Не указано'
            else: 
                df['Жилая площадь кв'][k] = find_living(soup).split()[0].replace(',', '.')
            if find_kitchen(soup) == 'Не указано':
                df['Площадь кухни'][k] = 'Не указано'
            else: 
                df['Площадь кухни'][k] = find_kitchen(soup).split()[0].replace(',', '.')
            if find_floor(soup) == 'Не указано':
                df['Этаж'][k] = 'Не указано'
            else:
                df['Этаж'][k] = (find_floor(soup).split()[0])
                          
            if find_floor(soup) == 'Не указано':
                df['Этажей в доме'][k] = 'Не указано'
            else: 
                df['Этажей в доме'][k] = find_floor(soup).split()[2]
            if find_finishing(soup) == 'Не указано':
                df['Отделка'][k] = 'Не указано'
            else:
                df['Отделка'][k] = find_finishing(soup)
            if find_year(soup) == 'Не указано':
                df['Год постройки'][k] = 'Не указано'
            else:
                df['Год постройки'][k] = find_year(soup)
            if type_floar(soup) == 'Не указано':
                df['Тип жилья'][k] = 'Не указано'
            else:
                df['Тип жилья'][k] = type_floar(soup)
            if type_san(soup) == 'Не указано':
                df['Санузел'][k] = 'Не указано'
            else:
                df['Санузел'][k] = type_san(soup)
            if type_bal(soup) == 'Не указано':
                df['Балкон'][k] = 'Не указано'
            else:
                df['Балкон'][k] = type_bal(soup)
            if type_rem(soup) == 'Не указано':
                df['Ремонт'][k] = 'Не указано'
            else:
                df['Ремонт'][k] = type_rem(soup)
            k += 1
    df_all = pd.concat([df_all,df])
    df_all = df_all.reset_index()

100%|██████████████████████████████████| 2/2 [12:59<00:00, 389.51s/it]


In [393]:
df_all = df_all.reset_index()

In [441]:
df_all

,index,id,Link,Date,Address,Location,Число комнат,Цена,Общая площадь,Жилая площадь кв,...,Этаж,Этажей в доме,Отделка,Расстояние до метро,Год постройки,Тип жилья,Санузел,Балкон,Ремонт,Цена за метр
0,0,0.0,https://www.cian.ru/sale/flat/279805008/,2022-11-06,"Москва, САО, р-н Молжаниновский, Синявинская у...",NaN,2,11150000,57.7,32.7,...,5,19,Не указано,NaN,Не указано,Вторичка,1 раздельный,1 лоджия,Евроремонт,193240.901213
1,1,1.0,https://www.cian.ru/sale/flat/279805007/,2022-11-06,"Москва, ЮЗАО, р-н Ломоносовский, ул. Гарибальд...",NaN,1,9700000,31.5,21.4,...,3,9,Не указано,18.0,Не указано,Вторичка,1 совмещенный,1 балкон,Не указано,307936.507937
2,2,2.0,https://www.cian.ru/sale/flat/279804992/,2022-11-06,"Москва, ВАО, р-н Гольяново, Байкальская ул., 5...",NaN,3,11900000,60,42,...,8,9,Не указано,3.0,1969,Вторичка,1 раздельный,2 балкона,Косметический,198333.333333
3,3,3.0,https://www.cian.ru/sale/flat/279804949/,2022-11-06,"Москва, ЮЗАО, р-н Академический, ул. Ивана Баб...",NaN,4,62000000,162.7,81,...,9,29,Не указано,5.0,2008,Вторичка,2 раздельных,1 лоджия,Дизайнерский,381069.452981
4,4,4.0,https://www.cian.ru/sale/flat/279804947/,2022-11-06,"Москва, ЮАО, р-н Даниловский, Шагал жилой комп...",NaN,3,47500000,104,Не указано,...,2,11,Без отделки,18.0,Не указано,Новостройка,Не указано,Не указано,Не указано,456730.769231
5,5,5.0,https://www.cian.ru/sale/flat/279804916/,2022-11-06,"Москва, ЮЗАО, р-н Академический, ул. Кржижанов...",NaN,2,24500000,64,36,...,1,5,Не указано,8.0,1956,Вторичка,1 раздельный,Не указано,Евроремонт,382812.5
6,6,6.0,https://www.cian.ru/sale/flat/279804915/,2022-11-06,"Москва, НАО (Новомосковский), Щербинка, ул. Сп...",NaN,1,8450000,42,10.5,...,2,14,Не указано,NaN,2003,Вторичка,1 совмещенный,1 балкон,Косметический,201190.47619
7,7,7.0,https://www.cian.ru/sale/flat/279804898/,2022-11-06,"Москва, ЮЗАО, р-н Черемушки, Новочеремушкинска...",NaN,1,23000000,55.6,13.7,...,19,22,Не указано,Не указано,2006,Вторичка,1 раздельный,1 балкон,Евроремонт,413669.064748
8,8,8.0,https://www.cian.ru/sale/flat/279804896/,2022-11-06,"Москва, ЮЗАО, р-н Академический, ул. Ивана Баб...",NaN,1,13350000,44.6,20.1,...,9,12,Не указано,7.0,1976,Вторичка,1 совмещенный,1 лоджия,Косметический,299327.35426
9,9,9.0,https://www.cian.ru/sale/flat/279804884/,2022-11-06,"Москва, СЗАО, р-н Северное Тушино, Туристская ...",NaN,1,13500000,40,Не указано,...,17,24,Не указано,15.0,2005,Вторичка,1 совмещенный,1 лоджия,Евроремонт,337500.0


In [397]:
df_all.to_excel('df_all.xlsx')

In [442]:
df_check

,Unnamed: 0,index,id,Link,Date,Address,Location,Число комнат,Цена,Общая площадь,...,Этаж,Этажей в доме,Отделка,Расстояние до метро,Год постройки,Тип жилья,Санузел,Балкон,Ремонт,Цена за метр
0,0,0,0,https://www.cian.ru/sale/flat/279805008/,2022-11-06,"Москва, САО, р-н Молжаниновский, Синявинская у...",NaN,2,11150000,57.7,...,5,19,Не указано,NaN,Не указано,Вторичка,1 раздельный,1 лоджия,Евроремонт,193240.901213
1,1,1,1,https://www.cian.ru/sale/flat/279805007/,2022-11-06,"Москва, ЮЗАО, р-н Ломоносовский, ул. Гарибальд...",NaN,1,9700000,31.5,...,3,9,Не указано,18,Не указано,Вторичка,1 совмещенный,1 балкон,Не указано,307936.507937
2,2,2,2,https://www.cian.ru/sale/flat/279804992/,2022-11-06,"Москва, ВАО, р-н Гольяново, Байкальская ул., 5...",NaN,3,11900000,60.0,...,8,9,Не указано,3,1969,Вторичка,1 раздельный,2 балкона,Косметический,198333.333333
3,3,3,3,https://www.cian.ru/sale/flat/279804949/,2022-11-06,"Москва, ЮЗАО, р-н Академический, ул. Ивана Баб...",NaN,4,62000000,162.7,...,9,29,Не указано,5,2008,Вторичка,2 раздельных,1 лоджия,Дизайнерский,381069.452981
4,4,4,4,https://www.cian.ru/sale/flat/279804947/,2022-11-06,"Москва, ЮАО, р-н Даниловский, Шагал жилой комп...",NaN,3,47500000,104.0,...,2,11,Без отделки,18,Не указано,Новостройка,Не указано,Не указано,Не указано,456730.769231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,51,23,23,https://www.cian.ru/sale/flat/279803771/,2022-11-06,"Москва, ЗАО, р-н Солнцево, Боровское ш., 2к6На...",NaN,2,21500000,68.9,...,13,26,Не указано,NaN,2018,Вторичка,2 раздельных,1 балкон,Евроремонт,312046.444122
52,52,24,24,https://www.cian.ru/sale/flat/279803770/,2022-11-06,"Москва, СЗАО, р-н Южное Тушино, Лодочная ул., ...",NaN,3,11000000,36.0,...,1,9,Не указано,Не указано,1963,Вторичка,1 совмещенный,Не указано,Евроремонт,305555.555556
53,53,25,25,https://www.cian.ru/sale/flat/279803766/,2022-11-06,"Москва, ЮЗАО, р-н Обручевский, ул. Архитектора...",NaN,1,28300000,71.5,...,4,16,Не указано,10,2001,Вторичка,1 совмещенный,Не указано,Дизайнерский,395804.195804
54,54,26,26,https://www.cian.ru/sale/flat/279803707/,2022-11-06,"Москва, ЦАО, р-н Красносельский, Верхняя Красн...",NaN,2,12900000,39.0,...,13,15,Не указано,9,1978,Вторичка,1 совмещенный,"1 балкон, 1 лоджия",Косметический,330769.230769


_______

In [209]:
url = 'https://spb.cian.ru/sale/suburban/263172915/'

In [210]:
driver = undetected_chromedriver.Chrome()

In [211]:
driver.get(url)

In [212]:
page_source = driver.page_source

In [213]:
soup = BeautifulSoup(page_source, 'lxml')

In [214]:
soup.find_all('div','a10a3f92e9--info-title--JWtIm')

[<div class="a10a3f92e9--info-title--JWtIm" data-testid="object-summary-description-title">Общая</div>,
 <div class="a10a3f92e9--info-title--JWtIm" data-testid="object-summary-description-title">Этаж</div>]

In [215]:
    temp_1 = soup.find_all('div', 'a10a3f92e9--info-value--bm3DC')
    temp_2 = soup.find_all('div','a10a3f92e9--info-title--JWtIm')

In [216]:
temp_1

[<div class="a10a3f92e9--info-value--bm3DC" data-testid="object-summary-description-value">120 м²</div>,
 <div class="a10a3f92e9--info-value--bm3DC" data-testid="object-summary-description-value">2 из 13</div>]

In [217]:
temp_2

[<div class="a10a3f92e9--info-title--JWtIm" data-testid="object-summary-description-title">Общая</div>,
 <div class="a10a3f92e9--info-title--JWtIm" data-testid="object-summary-description-title">Этаж</div>]

In [218]:
    temp_3 = soup.find_all('span', 'a10a3f92e9--name--x7_lt')
    temp_4 = soup.find_all('span','a10a3f92e9--value--Y34zN')

In [219]:
temp_3

[]

In [220]:
temp_4

[]

In [221]:
temp_5 = soup.find_all('span', 'a10a3f92e9--highway_distance--wUNBn')

In [222]:
temp_5[0].text

IndexError: list index out of range

In [223]:
temp_6 = soup.find_all('div', 'a10a3f92e9--container--RXoIe')

In [226]:
temp_6[0].text

'Объявление снято с публикации'

# Дата

In [168]:
get_date(soup)

'too old'

# Адрес

In [169]:
soup.find("address").text.strip().split(',')[0] == 'Москва'

False

In [170]:
get_address(soup)

('Лосино-Петровский городской округ',
 'Не указано',
 'ул. Михаила Марченко',
 '1')

# Число комнат

In [171]:
temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')

In [172]:
print(temp.text.split()[0].split('-')[0])

2


# Цена 


In [173]:
get_price(soup).replace('\xa0','').replace('₽', '')

'6800000'

# Общая площадь кв

In [174]:
temp = soup.find('h1', 'a10a3f92e9--title--UEAG3')

In [175]:
print(temp.text.split()[2].replace(',', '.'))

55.7


# Жилая площадь кв

In [176]:
if find_living(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_living(soup).split()[0].replace(',', '.'))

27


# Площадь кухни

In [177]:
if find_kitchen(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_kitchen(soup).split()[0].replace(',', '.'))

10


# Расстояние до метро

In [193]:
find_distance(soup)

'23 км от МКАД'

# Год постройки

In [178]:
if find_year(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_year(soup))

2010


# Этаж

In [179]:
if find_floor(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_floor(soup).split()[0])

10


# Сколько этажей в доме

In [180]:
if find_floor(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_floor(soup).split()[2])

14


# Отделка

In [181]:
if find_finishing(soup) == 'Не указано':
    print('Не указано')
else: 
    print(find_finishing(soup))

Не указано


# Тип жилья

In [182]:
if type_floar(soup) == 'Не указано':
    print('Не указано')
else: 
    print(type_floar(soup))

Вторичка


# Санузел

In [183]:
if type_san(soup) == 'Не указано':
    print('Не указано')
else: 
    print(type_san(soup))

1 раздельный


# Балкон

In [184]:
if type_bal(soup) == 'Не указано':
    print('Не указано')
else: 
    print(type_bal(soup))

1 лоджия


# Ремонт

In [185]:
if type_rem(soup) == 'Не указано':
    print('Не указано')
else: 
    print(type_rem(soup))

Евроремонт


________________